# Exercise 1: Business cycle correlations

For this exercise, you'll be using macroeconomic data from the folder `data/FRED`.

1.  There are seven decade-specific files named `FRED_monthly_19X0.csv` where `X` identifies the decade (`X` takes on the values 5, 6, 7, 8, 9, 0, 1). Write a loop that reads in all seven files as DataFrames and store them in a list.

    *Hint:* Recall from the lecture that you should use `pd.read_csv(..., parse_dates=['DATE'])` to automatically parse strings stored in the `DATE` column as dates.
2.  Use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) to concate these data sets into a single `DataFrame` and set the `DATE` column as the index.
3.  You realize that your data does not include GDP since this variable is only reported at quarterly frequency.
    Load the GDP data from the file `GDP.csv` and merge it with your monthly data using an _inner join_.
4.  You want to compute how (percent) changes of the variables in your data correlate with percent changes in GDP.

    1. Create a _new_ `DataFrame` which contains the percent changes in CPI and GDP (using 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html), 
    see also the last exercise in workshop 3),
    and the absolute changes for the remaining variables (using 
    [`diff()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html)).
    2.  Compute the correlation of the percent changes in GDP with the (percent) changes of all other variables (using [`corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)). What does the sign and magnitude of the correlation coefficient tell you?



In [76]:
import pandas as pd
DATA_PATH = '../../data/FRED'

# Store imported DataFrames in this list
data = []

for x in range(1950, 2011, 10):
    filename = f'{DATA_PATH}/FRED_monthly_{x}.csv'
    df = pd.read_csv(filename, parse_dates=['DATE'])
    data.append(df)

In [77]:
len(data)

7

### PART 2 ###

In [78]:
# Concatenate into a singel DataFrame

df = pd.concat(data, ignore_index=True)

In [79]:
# Set tje DATE column as the index
df = df.set_index('DATE')

df

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
2019-08-01,256.0,3.6,2.1,0.6,63.1
2019-09-01,256.4,3.5,2.0,0.3,63.2
2019-10-01,257.2,3.6,1.8,-0.0,63.3


In [80]:
# 3

fn2 = f'{DATA_PATH}/GDP.csv'
GDP = pd.read_csv(fn2, parse_dates=['DATE'])
GDP = GDP.set_index('DATE')


In [81]:
df = df.merge(GDP, on='DATE', how='inner')
df

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,GDP
DATE,,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9,2346.1
1950-04-01,23.6,5.8,NaN,NaN,59.2,2417.7
1950-07-01,24.1,5.0,NaN,NaN,59.1,2511.1
1950-10-01,24.5,4.2,NaN,NaN,59.4,2559.2
1951-01-01,25.4,3.7,NaN,NaN,59.1,2594.0
...,...,...,...,...,...,...
2018-10-01,252.8,3.8,2.2,-0.2,62.9,20304.9
2019-01-01,252.6,4.0,2.4,0.6,63.1,20431.6
2019-04-01,255.2,3.7,2.4,3.1,62.8,20602.3


### Part 4 ###

In [82]:
#Compute percent changes in CPI and GDP
df_changes = df[['CPI', 'GDP']].pct_change() * 1000.0


In [83]:
variables = ['UNRATE', 'FEDFUNDS', 'REALRATE', 'LFPART']

df_changes[variables] = df[variables].diff()

df_changes


,CPI,GDP,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,,
1950-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1950-04-01,4.255319,30.518733,-0.7,NaN,NaN,0.3
1950-07-01,21.186441,38.631757,-0.8,NaN,NaN,-0.1
1950-10-01,16.597510,19.154952,-0.8,NaN,NaN,0.3
1951-01-01,36.734694,13.597999,-0.5,NaN,NaN,-0.3
...,...,...,...,...,...,...
2018-10-01,6.369427,1.415453,0.0,0.3,-0.1,-0.1
2019-01-01,-0.791139,6.239873,0.2,0.2,0.8,0.2
2019-04-01,10.292953,8.354705,-0.3,0.0,2.5,-0.3


In [84]:
# Compute pairwise correlations
df_changes.corr().loc['GDP']

CPI        -0.113091
GDP         1.000000
UNRATE     -0.564872
FEDFUNDS    0.206370
REALRATE    0.074500
LFPART      0.019639
Name: GDP, dtype: float64

***
# Exercise 2: Loading many data files

In the previous exercise, you loaded the individual files by specifing an explicit list of file names. This can become tedious or infeasible if your data is spread across many files with varying file name patterns. Python offers the possibility to iterate over all files in a directory (for example, using [`os.listdir()`](https://docs.python.org/3/library/os.html#os.listdir)),
or to iterate over files that match a pattern, for example using [`glob.glob()`](https://docs.python.org/3/library/glob.html).

Repeat parts (1) and (2) from the previous exercise, but now iterate over the input files using 
[`glob.glob()`](https://docs.python.org/3/library/glob.html). You'll need to use a wildcard `*` and make sure to match only the relevant files in `data/FRED`, i.e., those that start with `FRED_monthly`.

In [90]:
pattern = f'{DATA_PATH}/FRED_monthly_[12]*.csv'

import glob

# List to hold imported data frame
data = []

for file in glob.glob(pattern):
    print(f'Processing file {file}')
    df = pd.read_csv(file)
    data.append(df)

Processing file ../../data/FRED/FRED_monthly_1950.csv
Processing file ../../data/FRED/FRED_monthly_1990.csv
Processing file ../../data/FRED/FRED_monthly_1980.csv
Processing file ../../data/FRED/FRED_monthly_2000.csv
Processing file ../../data/FRED/FRED_monthly_2010.csv
Processing file ../../data/FRED/FRED_monthly_1970.csv
Processing file ../../data/FRED/FRED_monthly_1960.csv


In [101]:
# Merge this to single DataFrame
df = pd.concat(data).sort_values('DATE').reset_index(drop=True)

In [102]:
df

,DATE,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
0,1950-01-01,23.5,6.5,NaN,NaN,58.9
1,1950-02-01,23.6,6.4,NaN,NaN,58.9
2,1950-03-01,23.6,6.3,NaN,NaN,58.8
3,1950-04-01,23.6,5.8,NaN,NaN,59.2
4,1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...,...
835,2019-08-01,256.0,3.6,2.1,0.6,63.1
836,2019-09-01,256.4,3.5,2.0,0.3,63.2
837,2019-10-01,257.2,3.6,1.8,-0.0,63.3
838,2019-11-01,257.9,3.6,1.6,-0.2,63.3


In [103]:
df.set_index('DATE').sort_index()

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
2019-08-01,256.0,3.6,2.1,0.6,63.1
2019-09-01,256.4,3.5,2.0,0.3,63.2
2019-10-01,257.2,3.6,1.8,-0.0,63.3


***
# Exercise 3: Decade averages of macro time series


For this exercise, you'll be using macroeconomic data from the folder `data/FRED`.

1.  There are five files containing monthly observations on annual inflation (INFLATION), the Fed Funds rate (FEDFUNDS), the labor force participation rate (LFPART), the 1-year real interest rate (REALRATE) and the unemployment rate (UNRATE).
    Write a loop to import these files and store the individual DataFrames in a list.

    Then use 
    [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
    to concatenate this list of DataFrames along the column dimension
    using an outer join (`join='outer'`) to obtain a merged data set.

    *Hint:* Recall from the lecture that you should use 
    `pd.read_csv(..., parse_dates=['DATE'], index_col='DATE')` to automatically parse strings stored in the `DATE` column as dates and set the `DATE`
    column as the index.

3.  You want to compute the average value of each variable by decade, but you want to include only decades without _any_ missing values for _all_ variables.

    1.  Create a variable `Decade` which stores the decade (1940, 1950, ...) for each observation.

        *Hint:* You should have set the `DATE` as the `DataFrame` index. Then you can access the calendar year using the attribute `df.index.year` which can be used to compute the decade.

    2.  Create an indicator variable which takes on the value `True` 
        whenever all observations for a given date are present, and `False`
        if at least one variable has a missing observation. 

    3.  Aggregate this indicator to decades using a
    [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) so that the indicator takes on the value `True` whenever
    _all_ variables in a given decade have no missing values, and `False`
    otherwise.

        *Hint:* You can use the 
        [`all()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.all.html) aggregation for this.

    4.  Merge this decade-level indicator data back into the original `DataFrame` (_many-to-one_ merge). 
4.  Using this indicator, drop all observations which are in a decade with missing values.
5.  Compute the decade average for each variable.

**Challenge**

-   Your pandas guru friend claims that all the steps in 2.2 to 2.4 can be done with a single one-liner using [`transform()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html). Can you come up with a solution?

    

In [ ]:
import glob
import pandas as pd

DATA_PATH2 = '../DATA/stockmarket'

stock = f'{DATA_PATH2}/'


***
# Exercise 4: Merging additional Titanic data

In this exercise, you'll be working with the the original Titanic data set in `titanic.csv` and additional (partly fictitious) information on passengers stored in `titanic-additional.csv`, both located in the `data/` folder.

The goal of the exercise is to calculate the survival rates by country of residence (for this exercise we restrict ourselves to the UK, so these will be England, Scotland, etc.).

1.  Load the `titanic.csv` and `titanic-additional.csv` into two DataFrames.

    Inspect the columns contained in both data sets. As you can see, the original data contains the full name including the title
    and potentially maiden name (for married women) in a single column.
    The additional data contains this information in separate columns.
    You want to merge these data sets, but you first need to create common keys in both DataFrames.

2.  Since the only common information is the name, you'll need to extract the individual name components from the original DataFrame
    and use these as merge keys.

    Focusing only on men (who have names that are much easier to parse), split the `Name` column into the tokens 
    `Title`, `FirstName` and `LastName`, just like the columns in the second DataFrame.

    *Hint:* This is the same task as in the last exercise in Workshop 2. You can just use your solution here.

3.  Merge the two data sets based on the columns `Title`, `FirstName` and `LastName` you just created using a _left join_ (_one-to-one_ merge).
    Tabulate the columns and the number of non-missing observations to make sure that merging worked. 

    *Note:* The additional data set contains address information only for passengers from the UK, so some of these fields will be missing.

4.  You are now in a position to merge the country of residence (_many-to-one_ merge). Load the country data from `UK_post_codes.csv` which contains 
    the UK post code prefix (which you can ignore), the corresponding city, and the corresponding country.

    Merge this data with your passenger data set using a _left join_ (what is the correct merge key?).

5.  Tabulate the number of observations by `Country`, including the number of observations with missing `Country` (these are passengers residing outside the UK).

    Finally, compute the mean survival rate by country.